In [5]:
%matplotlib inline

import numpy as np # NUMPY
import pandas as pd # PANDAS
import matplotlib.pyplot as plt # MATPLOTLIB
import seaborn as sns # SEABORN
import scipy
import sklearn
from sklearn.decomposition import PCA
import cv2
import os 
from skimage import filters
from scipy import ndimage

ModuleNotFoundError: No module named 'cv2'

In [6]:
1+1

2

In [ ]:
def get_landmarks_of_folder(files_vector):
    frames_landmarks_XYord = []
    for i in np.sort(files_vector):
        file_landmarks = get_landmarks_of_file(i)
        frames_landmarks_XYord.append(file_landmarks)
    frames_landmarks_XYord = np.asarray(frames_landmarks_XYord)
    return frames_landmarks_XYord

def get_landmarks_of_file(f):
    data = pd.read_csv(f, header = None)
    data.columns =['landmarks']
    quantity_of_landmarks = data.size #68
    data = data['landmarks'].str.split('   ', expand=True)
    data.columns =['landmarks', 'landmarkX', 'landmarkY']
    data =data.drop(columns=['landmarks'])
    landmarkXY= pd.concat([data['landmarkX'], data['landmarkY'] ], ignore_index=True)
    landmarkXY_array = landmarkXY.to_numpy(dtype=float)
    return landmarkXY_array
    

def testimport():
    print('estafuncando')    

In [ ]:

#Cambiamos a [x1, y1, x2,y2]
#Ahora lo hacemos en pares
def intercalate_datavector(vector):    
    x_coord = vector[0:int(len(vector)/2)]    
    y_coord = vector[int(len(vector)/2):len(vector)]
    output = []
    for (a, b) in zip(x_coord, y_coord):
        output.append(a)
        output.append(b)     
    return np.asarray(output)

def intercalate_data_of_datavectors(vectors, pairs):
    intercalated_data = []
    for i in range(0, len(vectors)):
        if (pairs == 0):
            intercalated = intercalate_datavector(vectors.iloc[i])
        else:
            intercalated = intercalate_datavector_as_pairs(vectors.iloc[i])
        intercalated_data.append(intercalated)
    intercalated_data = np.asarray(intercalated_data)
    return intercalated_data

        
def intercalate_data_of_vectors(vectors, pairs):
    intercalated_data = []
    for i in range(0, len(vectors)):
        if (pairs == 0):
            intercalated = intercalate_datavector(vectors[i])
        else:
            intercalated = intercalate_datavector_as_pairs(vectors[i])
        intercalated_data.append(intercalated)
    intercalated_data = np.asarray(intercalated_data)
    return intercalated_data 
        
def intercalate_datavector_as_pairs(vector):
    x_coord = vector[0:int(len(vector)/2)]    
    y_coord = vector[int(len(vector)/2):len(vector)]
    output = []
    for (a, b) in zip(x_coord, y_coord):
        output.append(np.array([a,b]))            
    return np.asarray(output)

In [ ]:

        
def opa(a, b):# a, b matrices de puntos [[],[],[]] a and b need to have same dimension
    aT = a.mean(0) # column mean [[1,2,3],[1,2,3]]<-(1,2,3)
    bT = b.mean(0)
    A = a - aT  #We rest aT and bT to center the matrix (translation)
    B = b - bT
    aS = np.sum(A * A)**.5 #to see the scale, we first calculate square of squared sum
    bS = np.sum(B * B)**.5
    A /= aS  #we standarize the scale.
    B /= bS
    U, _, V = np.linalg.svd(np.dot(B.T, A)) # we get the principal values of the mult for getting the rotation more close 
    aR = np.dot(U, V) #<-the rotation matrix is the multiplication of these 2
    if np.linalg.det(aR) < 0: # we rotate to the positive side
        V[1] *= -1
        aR = np.dot(U, V)
    aS = aS / bS  #to scale A most similar to B, we divide the scale by sSvale
    aT-= (bT.dot(aR) * aS) #to get the translation in B coordinates, rest b mean rotated to A, and scaled to A 
    aD = (np.sum((A - B.dot(aR))**2) / len(a))**.5  # the square distance (error)of the A and B rotated after scale and translation
    return aR, aS, aT, aD #return aROtation, aScale, aTranslation, Adifference
        
def gpa(v, n=-1): #v is vector of images, n indicates index to reflect
    if n < 0: 
        p = avg(v)
    else:
        p = v[n]
    l = len(v) #quantity of images
    r, s, t, d = np.ndarray((4, l), object) #4 array of len l, each a matrix
    for i in range(l):
        r[i], s[i], t[i], d[i] = opa(p, v[i]) 
    return r, s, t, d#<-calculates the matrix and the error for all images, you use the one with the smallest one (can iterate this)

def avg(v): #given a vector of images, calculates average shape of images 
    v_= np.copy(v)
    l = len(v_) 
    R, S, T = [list(np.zeros(l)) for _ in range(3)]
    for i, j in np.ndindex(l, l): #hace la daptacion de cada i para cada j
        r, s, t, _ = opa(v_[i], v_[j]) 
        R[j] += np.arccos(min(1, max(-1, np.trace(r[:1])))) * np.sign(r[1][0]) 
        S[j] += s 
        T[j] += t #ajusta rot, s y translacion en todo j con todo i
    for i in range(l):#hace el avg dado todo j
        a = R[i] / l
        r = [np.cos(a), -np.sin(a)], [np.sin(a), np.cos(a)]
        v_[i] = v_[i].dot(r) * (S[i] / l) + (T[i] / l) 
    return v_.mean(0)

In [ ]:
def align_and_transform_image(reference_image, reference_landmarks, landmarks, images):
    even = np.asarray(reference_landmarks[::2] )
    odds = np.asarray( reference_landmarks[1::2] )
    
    
    target_X_img = cv2.imread(reference_image,0)
    lan =[]
    for (a, b) in zip(even, odds):
        lan.append(np.asarray([a,b]))  
    
    
    even1 = np.asarray(landmarks[::2] )
    odds1 = np.asarray( landmarks[1::2] )
    lan1 =[]
    for (a, b) in zip(even1, odds1):
        lan1.append(np.asarray([a,b]))
    
    
    X_pts = np.asarray(lan)
    Y_pts = np.asarray(lan1)
    

    input_Y_img = cv2.imread(images)
    
    r,s,tr, d= opa(X_pts,Y_pts)
    # Build and apply transform matrix...
    # Note: for affine need 2x3 (a,b,c,d,e,f) form
    R = np.eye(3)
    R[0:2,0:2] = r
    S = np.eye(3) * s
    S[2,2] = 1
    t = np.eye(3)
    t[0:2,2] = tr
    M = np.dot(np.dot(R,S),t.T).T
    tr_Y_img = cv2.warpAffine(input_Y_img,M[0:2,:],(640,490))

    
    #transform deberia hacer el a[1] = a[1].dot(r) * s + t
    Z_pts = Y_pts.dot(r) * s + tr
    
    return tr_Y_img, Z_pts 
    


    

def align_and_transform_datavector_of_images(reference_image, reference_landmarks, landmarks, images, pathi):        
    transformed_landmarks_total = []        
    for i in range(0, len(landmarks)):
        print(images[i].split('/')[6])
        tr_Y_img, Z_pts = align_and_transform_image(reference_image, reference_landmarks, landmarks.iloc[i], images[i])        
        transformed_landmarks_total.append(Z_pts)
        filename = 'warped_image_procustres_'+ str(images[i].split('/')[6]) 
        path = pathi
        cv2.imwrite(os.path.join(path , filename), tr_Y_img)      
    transformed_landmarks_total = np.asarray(transformed_landmarks_total)
    return transformed_landmarks_total


def align_and_transform_vector_of_images(reference_image, reference_landmarks, landmarks, images, pathi):        
    transformed_landmarks_total = []        
    for i in range(0, len(landmarks)):
        tr_Y_img, Z_pts = align_and_transform_image(reference_image, reference_landmarks, landmarks[i], images[i])        
        transformed_landmarks_total.append(Z_pts)
        filename = 'warped_image_procustres_'+ str(images[i].split('/')[6]) 
        path = pathi
        cv2.imwrite(os.path.join(path , filename), tr_Y_img)      
    transformed_landmarks_total = np.asarray(transformed_landmarks_total)
    return transformed_landmarks_total

In [1]:
def get_first_frames_and_images(path,my_data):
    first_images = []
    first_frames = []
    total = 0

    current_neutral_index = 0
    images_names = np.sort(os.listdir(path))
    for i in range(0, len(images_names)):        
        if int(images_names[i].split('_')[5].split('.')[0]) == 1:
            first_frames.append(my_data.iloc[i])
            first_images.append(path +'/'+ images_names[i])

    first_frames = np.asarray( first_frames, dtype=object)
    first_images = np.asarray( first_images, dtype=object)

    return first_frames, first_images

def get_first_frames(path,my_data):
    return get_first_frames_and_images(path,my_data)[0]

def get_first_images(path,my_data):
    return get_first_frames_and_images(path,my_data)[1]

In [ ]:
def get_first_frames_and_images22(path):
    first_images = []
    first_frames = []
    total = 0
    
    current_neutral_index = 0
    images_names = np.sort(os.listdir(path))    
    for i in range(0, len(images_names)):        
        if int(images_names[i].split('_')[5].split('.')[0]) == 1:
            first_frames.append(my_data.iloc[i])
            first_images.append(path +'/'+ images_names[i])   

    first_frames = np.asarray( first_frames, dtype=object)   
    first_images = np.asarray( first_images, dtype=object) 
    
    return first_frames, first_images

def get_first_frames22(path):
    return get_first_frames_and_images(path)[0]
    
def get_first_images22(path):
    return get_first_frames_and_images(path)[1]

In [ ]:
#this fuction changes order [x1..xn,y1..yn] to [x1, y1...xn, yn]
def change_to_intercalate_order(frame_list):
    intercalated_first_frames = []
    for fframe in frame_list:
        xaxis = fframe[0: int(len(frame_list[0])/2)]
        yaxis = fframe[int(len(frame_list[0])/2):len(frame_list[0])]
        lan =[]
        for (a, b) in zip(xaxis, yaxis):
            lan.append(np.asarray([a,b]))        
        intercalated_first_frames.append( np.asarray(lan))
    intercalated_first_frames = np.asarray(intercalated_first_frames)
    return intercalated_first_frames


    

In [ ]:
from scipy.spatial import ConvexHull, convex_hull_plot_2d

def feature_point_one(im):
    point1 = im[18][0]
    point2 = im[25][0]
    return np.abs(point2-point1)

def feature_point_two(im):
    point1 = im[1][0]
    point2 = im[15][0]
    return np.abs(point2-point1)
   
def feature_point_three(im):
    point1 = im[1][0]
    point2 = im[14][0]
    return np.abs(point2-point1)

def feature_point_four(im):   
    point1a = im[3][0]
    point2a = im[13][0]        
    point1b = im[4][0]
    point2b = im[12][0]
    point1 = (point1a + point1b)/2#mean
    point2 = (point2a + point2b)/2
    return np.abs(point2-point1)

def feature_point_five(im):   
    point1 = im[6][0]
    point2 = im[10][0]    
    return np.abs(point2-point1)

def feature_point_six(im):
    point1 = im[31][0]
    point2 = im[35][0]
    return np.abs(point2-point1)

def feature_point_seven(im):
    point1 = im[61][0]
    point2 = im[65][0]
    return np.abs(point2-point1)

def feature_point_eight(im):
    point1a = im[37][0]
    point2a = im[43][0]        
    point1b = im[38][0]
    point2b = im[44][0]
    point1 = (point1a + point1b)/2#mean
    point2 = (point2a + point2b)/2
    return np.abs(point2-point1)

def feature_point_nine(im):
    point1 = im[42][0]
    point2 = im[45][0]
    return np.abs(point2-point1)

def feature_point_ten(im):
    point1 = im[39][0]
    point2 = im[42][0]
    return np.abs(point2-point1)

def feature_point_eleven(im):
    point1 = im[36][0]
    point2 = im[39][0]
    return np.abs(point2-point1)

def feature_point_twelve(im):
    point1 = im[27][1]
    point2 = im[33][1]
    return np.abs(point2-point1)

def feature_point_thirteen(im):
    point1 = im[33][1]
    point2 = im[8][1]
    return np.abs(point2-point1)

def feature_point_fourteen(im):
    point1a = im[38][1]
    point2a = im[20][1]        
    point1b = im[43][1]
    point2b = im[23][1]
    point1 = (point1a + point1b)/2#mean
    point2 = (point2a + point2b)/2
    return np.abs(point2-point1)

def feature_point_fifteen(im):
    point1 = im[37][1]
    point2 = im[41][1]        
    point3 = im[38][1]
    point4 = im[40][1]     
    point5 = im[43][1]
    point6 = im[47][1]        
    point7 = im[44][1]
    point8 = im[46][1]
    point_pair1 = np.abs(point2-point1)
    point_pair2 = np.abs(point4-point3)
    point_pair3 = np.abs(point6-point5)
    point_pair4 = np.abs(point8-point7)
    return np.mean(np.array([point_pair1, point_pair2, point_pair3, point_pair4]))
    
def feature_point_sixteen(im):
    point1 = im[33][1]
    point2 = im[56][1]
    return  np.abs(point2-point1)

def feature_point_seventeen(im):
    point1 = im[67][1]
    point2 = im[8][1]
    return  np.abs(point2-point1)

def feature_point_eighteen(im):
    points = im[39:59]
    hull = ConvexHull(points)   
    return  hull.volume

def feature_point_ninetotwentyone(im, imagen):
        #Rectangulo actual en anterorior era siempre imagen 0. FIjarsefirst_frame[0][1][0]
    left_cheek_x0 = int(im[1][0])#x de landmark 2
    left_cheek_x1 = int(im[31][0])#
    left_cheek_y0 = int(im[28][1])
    left_cheek_y1 = int(im[2][1])

    right_cheek_x0 = int(im[35][0])#x de landmark 2
    right_cheek_x1 = int(im[15][0])#
    right_cheek_y0 = int(im[28][1])
    right_cheek_y1 = int(im[14][1])
    
    img = cv2.imread(imagen)
    
    imgall = img[left_cheek_y0: right_cheek_y1,left_cheek_x0:right_cheek_x1 ]
    imgall =cv2.cvtColor(imgall, cv2.COLOR_BGR2GRAY)   
    nineteen, twenty, twentyone = cheeks_function_with_nose(imgall, left_cheek_y0,left_cheek_x0)
    return  nineteen, twenty, twentyone

def feature_point_nineteen(im, imagen):
    return feature_point_ninetotwentyone(im, imagen)[0]

def feature_point_twenty(im, imagen):
    return feature_point_ninetotwentyone(im, imagen)[1]

def feature_point_twentyone(im, imagen):
    return feature_point_ninetotwentyone(im, imagen)[2]



In [ ]:
feature_points_options = {1 : feature_point_one,
           2 : feature_point_two,
           3 : feature_point_three,
           4 : feature_point_four,
           5 : feature_point_five,
           6 : feature_point_six,
           7 : feature_point_seven,
           8 : feature_point_eight,
           9 : feature_point_nine,
           10 : feature_point_ten,
           11 : feature_point_eleven,
           12 : feature_point_twelve,
           13 : feature_point_thirteen,
           14 : feature_point_fourteen,
           15 : feature_point_fifteen,
           16 : feature_point_sixteen,
           17 : feature_point_seventeen,
           18 : feature_point_eighteen,
           19 : feature_point_nineteen,
           20 : feature_point_twenty,
           21 : feature_point_twentyone,
}

In [ ]:
   
def cheeks_function_with_nose(imgall, left_cheek_y0,left_cheek_x0):
    #usamos sobel para sacar las derivadas
    edgesx = cv2.Sobel(imgall, -1, dx=1, dy=0, ksize=1)
    edgesy = cv2.Sobel(imgall, -1, dx=0, dy=1, ksize=1)
#   edgesy = cv2.Sobel(imgleft, -1, dx=0, dy=1, ksize=1)    
    output = np.zeros((len(edgesx),len(edgesx[0])))
    
    for i in range(0, len(edgesx)):
        for j in range(0, len(edgesx[0])):
            output[i][j] = math.sqrt(pow(edgesx[i][j],2) + pow(edgesy[i][j],2)+ pow(-1,2)) #raiz de suma de cuadrados y -1 al cuadradp

    angle = np.zeros((len(edgesx),len(edgesx[0]),3))
    
    for i in range(0, len(edgesx)):
        for j in range(0, len(edgesx[0])):
            pos =np.array([edgesx[i][j], edgesy[i][j], -1]/output[i][j])
            angle[i][j] = pos
            
    beta = np.array([0,0,1])    

    #dados los angles, sacamos los coefficientes
        
    coff = np.zeros((len(edgesx),len(edgesx[0])))
    for i in range(0, len(edgesx)):
        for j in range(0, len(edgesx[0])):
            coff[i][j] = 1- np.dot(angle[i][j],beta)

    
    coff = coff.flatten()
    coff_sum = coff.sum() #lo usamos para center of mass
    coff_average = coff.mean()#es el 19
    
    partial_sumx = []    
    for i in range(0, len(edgesx)):#losy0debencambiarse arriba
        partial_sumx.append(coff[i]*(left_cheek_y0+i))
    
    partial_sumy = []
    for i in range(0, len(edgesx[0])):#losy0debencambiarse arriba
        partial_sumy.append(coff[len(edgesx)+i]*(left_cheek_x0+i))  
    

  
    center_of_massx = np.asarray(partial_sumx).sum()/coff_sum
    center_of_massy = np.asarray(partial_sumy).sum()/coff_sum

      
    return coff_average, center_of_massx, center_of_massy  

In [ ]:
def get_feature_point(im, number, imagen):
    if number == 19 or number == 20 or number == 21 :
        res = feature_points_options[number](im, imagen)
    else:
        res = feature_points_options[number](im)
    return res

def list_get_feature_number(lista, number, imagenlist):
    res = []
    for i in range(0, len(lista)):
        fea_p = get_feature_point(lista[i], number, imagenlist[i])
        res.append(fea_p)
    return np.asarray(res,dtype= object)
    
def list_get_feature_point(lista, number, imagenlist):
    res = list_get_feature_number(lista, number, imagenlist)
    return res


In [3]:
def do_the_structure_points_and_stack(first_frame, first_images):
  first_feature_point = list_get_feature_point(first_frame, 1, first_images)
  second_feature_point = list_get_feature_point(first_frame, 2, first_images)
  third_feature_point = list_get_feature_point(first_frame, 3, first_images)
  fourth_feature_point = list_get_feature_point(first_frame, 4, first_images)
  five_feature_point = list_get_feature_point(first_frame, 5, first_images)
  six_feature_point = list_get_feature_point(first_frame, 6, first_images)
  seven_feature_point = list_get_feature_point(first_frame, 7, first_images)
  eight_feature_point = list_get_feature_point(first_frame, 8, first_images)
  nine_feature_point = list_get_feature_point(first_frame, 9, first_images)
  ten_feature_point = list_get_feature_point(first_frame, 10, first_images)
  eleven_feature_point = list_get_feature_point(first_frame, 11, first_images)
  twelve_feature_point = list_get_feature_point(first_frame, 12, first_images)
  thirteen_feature_point = list_get_feature_point(first_frame, 13, first_images)
  fourteen_feature_point = list_get_feature_point(first_frame, 14, first_images)
  fifteen_feature_point = list_get_feature_point(first_frame, 15, first_images)
  sixteen_feature_point = list_get_feature_point(first_frame, 16, first_images)
  seventeen_feature_point = list_get_feature_point(first_frame, 17, first_images)
  eigtheen_feature_point = list_get_feature_point(first_frame, 18, first_images)
  nineteen_feature = list_get_feature_point(first_frame, 19, first_images)
  twenty_feature = list_get_feature_point(first_frame, 20, first_images)
  twentyone_feature = list_get_feature_point(first_frame, 21, first_images)

  structure_vector = first_feature_point
  structure_vector = np.vstack((structure_vector, second_feature_point))
  structure_vector = np.vstack((structure_vector, third_feature_point))
  structure_vector = np.vstack((structure_vector, fourth_feature_point))
  structure_vector = np.vstack((structure_vector, five_feature_point))
  structure_vector = np.vstack((structure_vector, six_feature_point))
  structure_vector = np.vstack((structure_vector, seven_feature_point))
  structure_vector = np.vstack((structure_vector, eight_feature_point))
  structure_vector = np.vstack((structure_vector, nine_feature_point))
  structure_vector = np.vstack((structure_vector, ten_feature_point))
  structure_vector = np.vstack((structure_vector, eleven_feature_point))
  structure_vector = np.vstack((structure_vector, twelve_feature_point))
  structure_vector = np.vstack((structure_vector, thirteen_feature_point))
  structure_vector = np.vstack((structure_vector, fourteen_feature_point))
  structure_vector = np.vstack((structure_vector, fifteen_feature_point))
  structure_vector = np.vstack((structure_vector, sixteen_feature_point))
  structure_vector = np.vstack((structure_vector, seventeen_feature_point))
  structure_vector = np.vstack((structure_vector, eigtheen_feature_point))
  structure_vector = np.vstack((structure_vector, nineteen_feature))
  structure_vector = np.vstack((structure_vector, twenty_feature))
  structure_vector = np.vstack((structure_vector, twentyone_feature))

  return structure_vector

In [ ]:
from skimage import filters

def ComputeImageGradient(image, way):
    if way == 0:
        GradX = cv2.Sobel(image, ddepth=-1, dx=1, dy=0, borderType=cv2.BORDER_REFLECT)
        GradY = cv2.Sobel(image, ddepth=-1, dx=0, dy=1, borderType=cv2.BORDER_REFLECT)
        GradX[-1,:] = 0
        GradY[:, -1] = 0 # Neumann boundary Condition
    elif way == 1:
        GradX, GradY = np.gradient(image)      
        GradX[-1,:] = 0
        GradY[:, -1] = 0 # Neumann boundary Condition     
    elif way == 2: 
        Kx = -1*np.array([[-1,0,1]])
        GradX = ndimage.convolve(img, Kx)
        Ky = -1*np.array([[-1],[0],[1]])
        GradY = ndimage.convolve(img, Ky)
        GradX[-1,:] = 0
        GradY[:, -1] = 0 # Neumann boundary Condition  
    else:
        print('not legal way')
        GradY = np.zeros(image.shape)
        GradX = np.zeros(image.shape)
    return GradX, GradY


#ComputeImageGradient(img,  0)
#gradx



In [ ]:
from numpy import linalg as LA
def ProximalF_Star(img_p1,img_p2):
    inp1 = img_p1.flatten()    
    inp2 = img_p2.flatten()# (313600,1)
    p_total =  np.dstack((inp1, inp2)) # queda (1,313600,2)
    p_total = p_total[0]

    partial = np.zeros(len(p_total))
    
    partial = np.asarray(list(map(LA.norm, p_total))) # cada el de p_total la norma 2
    partial = np.asarray(list(map(lambda e: max(1.0,e), partial)))
    
    partial = partial.reshape(img_p1.shape)
    
    output_img_p1 = img_p1 / partial
    output_img_p2 = img_p1 / partial 


    return output_img_p1, output_img_p2


def ProximalG(img_u, img_g, landa, tau):
    resta = img_u - img_g    
    output_image = np.zeros(img_u.shape)
    
    maskH = resta > landa*tau
    maskL = resta < -1*(landa*tau)
    maskB = np.abs(resta) <= landa*tau 

    lander = np.zeros(img_u.shape)
    lander.fill(landa*tau)
    
    landerA = lander *maskH
    imageA = img_u * maskH
    landerB = lander * maskL
    imageB = img_u * maskL    
   
    
    imageA = imageA - landerA
    imageB = imageB + landerB
    imageC = img_g * maskB
    
    output_image  = imageA +imageB + imageC    

    return output_image

def ProximalGD(img_u, img_g, landa, tau, resta):
    #resta = img_u - img_g    
    output_image = np.zeros(img_u.shape)
    for i in range(0, len(img_u)):
        for j in range(0, len(img_u[i])):            
            if resta[i][j] > landa*tau :
                output_image[i][j] = img_u[i][j] - (landa*tau)
            elif resta[i][j] < -1*(landa*tau):
                output_image[i][j] = img_u[i][j] + (landa*tau)
            else: # si el modulo de la resta es menor a landa*tau
                output_image[i][j] = img_g[i][j]
    return output_image

def ProximalGL(img_u, img_g, landa, tau):
    resta = img_u - img_g    
    output_image = np.zeros(img_u.shape)
    for i in range(0, len(img_u)):
        for j in range(0, len(img_u[i])):            
            if resta[i][j] > landa*tau :
                output_image[i][j] = img_u[i][j] - (landa*tau)
            elif resta[i][j] < -1*(landa*tau):
                output_image[i][j] = img_u[i][j] + (landa*tau)
            else: # si el modulo de la resta es menor a landa*tau
                output_image[i][j] = img_g[i][j]
    return output_image

In [ ]:
def ComputeDivergenceL(img_p1, img_p2, way=1, sp=[1,1]):
    if way == 0 :
        p1_dx =  cv2.Sobel(img_p1, ddepth=-1, dx=1, dy=0, borderType=cv2.BORDER_REFLECT)
        p2_dy =  cv2.Sobel(img_p2, ddepth=-1, dx=0, dy=1, borderType=cv2.BORDER_REFLECT)
        output_image = p1_dx + p2_dy
    elif way == 1 :
        width, height =img_p1.shape
        inp1 = img_p1.flatten()
        inp2 = img_p2.flatten()#no se si f o c
        vector_field = np.array([inp1, inp2])
        num_dims = 2        
        output_image = np.ufunc.reduce(np.add, [np.gradient(vector_field[i], sp[i], axis =0) for i in range(num_dims)]) 
        output_image = output_image.reshape(width,height)        
    return output_image

In [ ]:
def Chambolle_TV_L1(input_image, nb_iter, tau, sigma, landa, tetha, way):
    # inicializacion
    width, height  = input_image.shape
    size_image = width * height
    
    u = input_image    
    u_old = np.zeros(input_image.shape)    
    p1 =  np.zeros(input_image.shape)# p = (p1, p2) o (px, py)
    p2= np.zeros(input_image.shape)
    img1 = np.zeros(input_image.shape)
    img2 = np.zeros(input_image.shape)
    GradX = np.zeros(input_image.shape)
    GradY = np.zeros(input_image.shape)
    
    E = float('inf')
    
        
    for it in range (1, nb_iter):
        u_old = u
        GradX, GradY = ComputeImageGradient(u, way) 
        im1 = p1+ sigma*GradX
        im2 = p2 + sigma*GradY
        p1, p2 = ProximalF_Star(im1,im2)        
        div = ComputeDivergenceL(p1, p2)        
        image1 = u + tau*div
        u = ProximalG(image1, input_image, landa, tau)
        u = u + tetha* (u - u_old)
        
        if( it % 10 == 0):
            E_old = E
            E = 0
            GradX, GradY = ComputeImageGradient(u, way)
            E += np.sum(landa * abs(u-input_image) + np.sqrt(GradX**2 + GradY**2))
            E = E/size_image
            if(abs(E-E_old) < 0.001): break
    
    return u


In [ ]:
def cartoonTexture_grey(input_image, landa, nb_iter_max=1000, tau=0.35, sigma=0.35, tetha =1.0, way =3):
    
    
    cartoon = np.zeros(input_image.shape)    
    cartoon = Chambolle_TV_L1(input_image, nb_iter_max, tau, sigma, landa, tetha, way)
    
    GradX = np.zeros(input_image.shape) 
    GradY = np.zeros(input_image.shape) 
    
    BV_norm = 0.0
    
    GradX, GradY = ComputeImageGradient(cartoon, way)
    
    BV_norm += np.sum((GradX **2 + GradY **2)**(1/2))     
    
    
    vLim = 20.0
    
    texture = np.zeros(input_image.shape)
    
    texture = input_image - cartoon
    
    texture  = (texture + vLim )*255.0 /(2.0* vLim)
    maskL = texture  >= 0.0 # para que quede 0 cuando es menor a 0
    texture = texture * maskL
    texture[texture >255.0] =255.0 
      
    return cartoon, texture
    


In [ ]:
1+1